# Q1. Bird Image Classification with Transfer Learning
### Tóm tắt vấn đề: Phân loại hình ảnh các loại chim bằng CNN Transfer Learning
Cho tập data Bird Species gồm ~84k ảnh của 525 nhãn, mỗi nhãn là tên khoa học của 1 loại chim tại
Kaggle: https://www.kaggle.com/datasets/gpiosenka/100-bird-species.
### Yêu cầu:
#### 1. Sử dụng các kiến thức đã học (CNN, Transfer Learning), tạo mới hoặc sử dụng 1 mô hình pretrained phù hợp để phân loại ảnh của 525 nhãn trong tập data trên.
#### 2. Huấn luyện và tuning, tăng cường mô hình sử dụng các kỹ thuật đã học (Adam, LRScheduler, Data Augmentation…) trên tập data trên. Report lại performance trên tập Test. Nhận xét kết quả thu được.

## Bird Species Classification Using Transfer Learning

### 1. Load the data

In [1]:
import os

train_paths = []
train_labels = []
for bird_type in os.listdir("/kaggle/input/100-bird-species/train"):
    train_path_data = os.listdir(os.path.join("/kaggle/input/100-bird-species/train",bird_type))
    train_cur_path = os.path.join("/kaggle/input/100-bird-species/train",bird_type)
    train_paths.extend([os.path.join(train_cur_path,img) for img in train_path_data])
    train_labels.extend([bird_type]*len(train_path_data))

In [2]:
test_paths = []
test_labels = []
for bird_type in os.listdir("/kaggle/input/100-bird-species/test"):
    test_path_data = os.listdir(os.path.join("/kaggle/input/100-bird-species/test",bird_type))
    test_cur_path = os.path.join("/kaggle/input/100-bird-species/test",bird_type)
    test_paths.extend([os.path.join(test_cur_path,img) for img in test_path_data])
    test_labels.extend([bird_type]*len(test_path_data))

In [3]:
val_paths = []
val_labels = []
for bird_type in os.listdir("/kaggle/input/100-bird-species/valid"):
    val_path_data = os.listdir(os.path.join("/kaggle/input/100-bird-species/valid",bird_type))
    val_cur_path = os.path.join("/kaggle/input/100-bird-species/valid",bird_type)
    val_paths.extend([os.path.join(val_cur_path,img) for img in val_path_data])
    val_labels.extend([bird_type]*len(val_path_data))

In [4]:
import pandas as pd

In [5]:
train_data = pd.DataFrame({"path":train_paths,"label":train_labels})
test_data = pd.DataFrame({"path":test_paths,"label":test_labels})
val_data = pd.DataFrame({"path":val_paths,"label":val_labels})

### 2. Data Loader and Custom Data Class

In [6]:
from PIL import Image
import os
import torch
from torch.utils.data import DataLoader

# define a data class
class ClassificationDataset:
    def __init__(self, data, data_path, transform, training=True):
        """Define the dataset for classification problems

        Args:
            data ([dataframe]): [a dataframe that contain 2 columns: image name and label]
            data_path ([str]): [path/to/folder that contains image file]
            transform : [augmentation methods and transformation of images]
            training (bool, optional): []. Defaults to True.
        """
        self.data = data
        self.imgs = data["path"].unique().tolist()
        self.data_path = data_path
        self.training = training
        self.transform = transform

    def __getitem__(self, idx):
        img = Image.open(os.path.join(self.data_path, self.data.iloc[idx, 0]))
        label = self.data.iloc[idx, 1]
        if self.transform is not None:
            img = self.transform(img)
        return img, label

    def __len__(self):
        return len(self.imgs)

### Define Data Transform Strategy

In [7]:
import torchvision.transforms as transform
import torchvision
train_transform = transform.Compose([
                           transform.Resize((224, 224)),
                           transform.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
                           transform.RandomRotation(5),
                           transform.RandomAffine(degrees=11, translate=(0.1,0.1), scale=(0.8,0.8)),
                           transform.ToTensor(),
                           transform.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [8]:
val_transform = transform.Compose([
                           transform.Resize((224, 224)),
                           transform.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
                           transform.RandomRotation(5),
                           transform.RandomAffine(degrees=11, translate=(0.1,0.1), scale=(0.8,0.8)),
                           transform.ToTensor(),
                           transform.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [9]:
test_transform = transform.Compose([
                           transform.Resize((224, 224)),
                           transform.ToTensor(),
                           transform.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [10]:
## mapping data
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
train_data.label = le.fit_transform(train_data.label)
test_data.label = le.fit_transform(test_data.label)
val_data.label = le.fit_transform(val_data.label)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### implement data loaders for train/val/test

In [11]:
trainset = ClassificationDataset(train_data,data_path = "",transform=train_transform,training=True)
valset = ClassificationDataset(val_data,data_path = "",transform=val_transform,training=True)

In [12]:
testset = ClassificationDataset(test_data,data_path = "",transform=test_transform,training=False)

In [13]:
train_loader = DataLoader(trainset, batch_size=32, shuffle=True,)
val_loader = DataLoader(valset, batch_size=32, shuffle=True,)
test_loader = DataLoader(testset, batch_size=1, shuffle=False)

## 3. Define Metrics and Optimizers and Loss function

### CrossEntropyLoss

In [14]:
from sklearn import metrics as skmetrics
import numpy
class Metrics:
    def __init__(self, metric_names):
        self.metric_names = metric_names
        # initialize a metric dictionary
        self.metric_dict = {metric_name: [0] for metric_name in self.metric_names}

    def step(self, labels, preds):
        for metric in self.metric_names:
            # get the metric function
            do_metric = getattr(
                skmetrics, metric, "The metric {} is not implemented".format(metric)
            )
            # check if metric require average method, if yes set to 'micro' or 'macro' or 'None'
            try:
                self.metric_dict[metric].append(
                    do_metric(labels, preds, average="macro")
                )
            except:
                self.metric_dict[metric].append(do_metric(labels, preds))

    def epoch(self):
        # calculate metrics for an entire epoch
        avg = [sum(metric) / (len(metric) - 1) for metric in self.metric_dict.values()]
        metric_as_dict = dict(zip(self.metric_names, avg))
        return metric_as_dict

    def last_step_metrics(self):
        # return metrics of last steps
        values = [self.metric_dict[metric][-1] for metric in self.metric_names]
        metric_as_dict = dict(zip(self.metric_names, values))
        return metric_as_dict

In [15]:
train_metrics = Metrics(["accuracy_score","f1_score"])
val_metrics = Metrics(["accuracy_score","f1_score"])

In [16]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from torch import nn
criterion = nn.CrossEntropyLoss()

In [17]:
device

device(type='cuda')

## 4. Define the Model: Transfer Learning

In [18]:
from torchvision import models
from torch import nn
model = models.efficientnet_b0(pretrained=True).cuda()
for param in model.parameters():
    param.requires_grad = False
classifier = nn.Sequential(
    nn.Linear(in_features=model.classifier[-1].in_features, out_features=525,bias=True),
)
model.classifier  = classifier

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-3dd342df.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 62.6MB/s]


In [19]:
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, "min", patience=2, factor=0.5
    )

## 5. Define a training epoch

In [20]:
model = model.to(device)

In [21]:
def train_one_epoch(
    model,
    train_loader,
    test_loader,
    device,
    optimizer,
    criterion,
    train_metrics,
    val_metrics,
):

    # training-the-model
    train_loss = 0
    valid_loss = 0
    all_labels = []
    all_preds = []
    model.train()
    for data, target in train_loader:
        # move-tensors-to-GPU
        data = data.type(torch.FloatTensor).to(device)
        # target=torch.Tensor(target)
        target = target.float().to(device)
        # clear-the-gradients-of-all-optimized-variables
        optimizer.zero_grad()
        # forward-pass: compute-predicted-outputs-by-passing-inputs-to-the-model
        output = model(data)
        # get the prediction label and target label
        output = model(data)
        preds = torch.argmax(output, axis=1).cpu().detach().numpy()
        labels = target.cpu().numpy()
        # calculate-the-batch-loss
        loss = criterion(output.type(torch.FloatTensor), target.type(torch.LongTensor))
        # backward-pass: compute-gradient-of-the-loss-wrt-model-parameters
        loss.backward()
        # perform-a-ingle-optimization-step (parameter-update)
        optimizer.step()
        # update-training-loss
        train_loss += loss.item() * data.size(0)
        # calculate training metrics
        all_labels.extend(labels)
        all_preds.extend(preds)
    
    train_metrics.step(all_labels, all_preds)

    # validate-the-model
    model.eval()
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for data, target in test_loader:
            data = data.type(torch.FloatTensor).to(device)
            target = target.to(device)
            output = model(data)
            preds = torch.argmax(output, axis=1).tolist()
            labels = target.tolist()
            all_labels.extend(labels)
            all_preds.extend(preds)
            loss = criterion(output, target)

            # update-average-validation-loss
            valid_loss += loss.item() * data.size(0)

    val_metrics.step(all_labels, all_preds)
    train_loss = train_loss / len(train_loader.sampler)
    valid_loss = valid_loss / len(test_loader.sampler)

    return (
        train_loss,
        valid_loss,
        train_metrics.last_step_metrics(),
        val_metrics.last_step_metrics(),
    )

## 6. Begin Training

In [22]:
from tqdm import tqdm

In [23]:

from datetime import datetime
time_str = str(datetime.now().strftime("%Y%m%d-%H%M"))

In [24]:
num_epoch = 10
best_val_acc = 0
import logging
import numpy as np
print("begin training process")
for i in tqdm(range(0, num_epoch)):
    loss, val_loss, train_result, val_result = train_one_epoch(
        model,
        train_loader,
        val_loader,
        device,
        optimizer,
        criterion,
        train_metrics,
        val_metrics,
    )

    scheduler.step(val_loss)
    print(
        "Epoch {} / {} \n Training loss: {} - Other training metrics: ".format(
            i + 1, num_epoch, loss
        )
    )
    print(train_result)
    print(
        " \n Validation loss : {} - Other validation metrics:".format(val_loss)
    )
    print(val_result)
    print("\n")
    # saving epoch with best validation accuracy
    if best_val_acc < float(val_result["accuracy_score"]):
        print(
            "Validation accuracy= "+
            str(val_result["accuracy_score"])+
            "===> Save best epoch"
        )
        best_val_acc = val_result["accuracy_score"]
        torch.save(
            model.state_dict(),
            "./" +  "best.pt",
        )
    else:
        print(
            "Validation accuracy= "+ str(val_result["accuracy_score"])+ "===> No saving"
        )
        continue

begin training process


 10%|█         | 1/10 [16:29<2:28:27, 989.68s/it]

Epoch 1 / 10 
 Training loss: 2.4985354363435888 - Other training metrics: 
{'accuracy_score': 0.5282448159744787, 'f1_score': 0.5241055037870551}
 
 Validation loss : 1.066960719335647 - Other validation metrics:
{'accuracy_score': 0.7683809523809524, 'f1_score': 0.7582305497179447}


Validation accuracy= 0.7683809523809524===> Save best epoch


 20%|██        | 2/10 [26:28<1:41:18, 759.82s/it]

Epoch 2 / 10 
 Training loss: 1.3329890718742912 - Other training metrics: 
{'accuracy_score': 0.6952797306079045, 'f1_score': 0.6935553391221384}
 
 Validation loss : 0.8923193124816531 - Other validation metrics:
{'accuracy_score': 0.792, 'f1_score': 0.7849295712068821}


Validation accuracy= 0.792===> Save best epoch


...

Epoch 8 / 10 
 Training loss: 0.7812834901501792 - Other training metrics: 
{'accuracy_score': 0.7996691676020559, 'f1_score': 0.7993888542715392}
 
 Validation loss : 0.7124446086883545 - Other validation metrics:
{'accuracy_score': 0.8201904761904761, 'f1_score': 0.8127609057609056}


Validation accuracy= 0.8201904761904761===> No saving


 90%|█████████ | 9/10 [1:37:47<10:27, 627.20s/it]

Epoch 9 / 10 
 Training loss: 0.6881558929549166 - Other training metrics: 
{'accuracy_score': 0.8208306256276954, 'f1_score': 0.8210913557952626}
 
 Validation loss : 0.6671276077315921 - Other validation metrics:
{'accuracy_score': 0.8365714285714285, 'f1_score': 0.8299111576254433}


Validation accuracy= 0.8365714285714285===> Save best epoch


100%|██████████| 10/10 [1:48:36<00:00, 651.66s/it]

Epoch 10 / 10 
 Training loss: 0.6602277630855239 - Other training metrics: 
{'accuracy_score': 0.828900573048975, 'f1_score': 0.8289074428039238}
 
 Validation loss : 0.6461658118764559 - Other validation metrics:
{'accuracy_score': 0.8316190476190476, 'f1_score': 0.8254122755551326}


Validation accuracy= 0.8316190476190476===> No saving


### Nhận xét: dùng Transfer Learning từ efficientnet_b0
-10 epoch đầu, chỉ unfreeze classifier với mục đích là để model học được cách nhận diện ảnh với feature extraction block cố định. Validation accuracy= ~84% (Save best epoch ở accuracy= 0.8365714285714285)

=> weight trên imagenet chưa tối ưu, có thể cải thiện

In [25]:
for param in model.parameters():
    param.requires_grad = True
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)

model = model.to(device)
num_epoch = 10
best_val_acc = 0.8528
import logging
import numpy as np
print("begin training process")
for i in tqdm(range(0, num_epoch)):
    loss, val_loss, train_result, val_result = train_one_epoch(
        model,
        train_loader,
        val_loader,
        device,
        optimizer,
        criterion,
        train_metrics,
        val_metrics,
    )

    scheduler.step(val_loss)
    print(
        "Epoch {} / {} \n Training loss: {} - Other training metrics: ".format(
            i + 1, num_epoch, loss
        )
    )
    print(train_result)
    print(
        " \n Validation loss : {} - Other validation metrics:".format(val_loss)
    )
    print(val_result)
    print("\n")
    # saving epoch with best validation accuracy
    if best_val_acc < float(val_result["accuracy_score"]):
        print(
            "Validation accuracy= "+
            str(val_result["accuracy_score"])+
            "===> Save best epoch"
        )
        best_val_acc = val_result["accuracy_score"]
        torch.save(
            model,
            "./" +  "best.pt"
        )
    else:
        print(
            "Validation accuracy= "+ str(val_result["accuracy_score"])+ "===> No saving"
        )
        continue

begin training process


 10%|█         | 1/10 [11:25<1:42:46, 685.14s/it]

Epoch 1 / 10 
 Training loss: 0.26943766616965775 - Other training metrics: 
{'accuracy_score': 0.9274413658651858, 'f1_score': 0.927585358664559}
 
 Validation loss : 0.18142078641482762 - Other validation metrics:
{'accuracy_score': 0.9539047619047619, 'f1_score': 0.9529228020656592}


Validation accuracy= 0.9539047619047619===> Save best epoch


 20%|██        | 2/10 [22:59<1:32:05, 690.68s/it]

Epoch 2 / 10 
 Training loss: 0.1441554370685006 - Other training metrics: 
{'accuracy_score': 0.9599456489631949, 'f1_score': 0.9599481329654627}
 
 Validation loss : 0.16265125668615135 - Other validation metrics:
{'accuracy_score': 0.9554285714285714, 'f1_score': 0.9544091569805856}


Validation accuracy= 0.9554285714285714===> Save best epoch


...

Epoch 8 / 10 
 Training loss: 0.03604387393380927 - Other training metrics: 
{'accuracy_score': 0.9899686890766232, 'f1_score': 0.9899586454585805}
 
 Validation loss : 0.1319780410045651 - Other validation metrics:
{'accuracy_score': 0.9664761904761905, 'f1_score': 0.9659169930598502}


Validation accuracy= 0.9664761904761905===> Save best epoch


 90%|█████████ | 9/10 [1:44:38<11:39, 699.30s/it]

Epoch 9 / 10 
 Training loss: 0.03350210360416372 - Other training metrics: 
{'accuracy_score': 0.9901931824895138, 'f1_score': 0.9902336993066726}
 
 Validation loss : 0.13740519412039293 - Other validation metrics:
{'accuracy_score': 0.9668571428571429, 'f1_score': 0.9665024393595822}


Validation accuracy= 0.9668571428571429===> Save best epoch


100%|██████████| 10/10 [1:55:41<00:00, 694.16s/it]

Epoch 10 / 10 
 Training loss: 0.029856514685946873 - Other training metrics: 
{'accuracy_score': 0.9915755892952088, 'f1_score': 0.9916171117953463}
 
 Validation loss : 0.12910669901842872 - Other validation metrics:
{'accuracy_score': 0.9687619047619047, 'f1_score': 0.9681766064623207}


Validation accuracy= 0.9687619047619047===> Save best epoch


### Nhận xét:
Unfreeze toàn bộ weight và dùng kết quả 10 epoch đầu như là init weight để tiết kiệm thời gian, cho phép model học dc bộ weight tối ưu trên data hiện tại: Validation accuracy= ~97% (Save best epoch ở accuracy= 0.9687619047619047)

=> mô hình phù hợp với bộ dữ liệu

## 7. Test the results

In [26]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [27]:
import copy
test_model = torch.load("/kaggle/working/best.pt")
test_model = test_model.to(device)

In [28]:
def test_result(model, test_loader, device):
    # testing the model by turning model "Eval" mode
    model.eval()
    preds = []
    labels = []
    with torch.no_grad():
        for data, target in test_loader:
            # move-tensors-to-GPU
            data = data.to(device)
            target = target.to(device)
            # forward-pass: compute-predicted-outputs-by-passing-inputs-to-the-model
            output = model(data)
            prob = nn.Softmax(dim=1)
            # applying Softmax to results
            probs = prob(output)
            labels.extend(target.tolist())
            preds.extend(torch.argmax(probs, axis=1).tolist())
    return labels,preds


In [29]:
labels,preds =test_result(test_model, test_loader, device)

In [30]:
report = classification_report(labels, preds, digits=4,target_names=le.classes_)

cm = confusion_matrix(labels, preds)

In [31]:
print(report)

                               precision    recall  f1-score   support

              ABBOTTS BABBLER     1.0000    1.0000    1.0000         5
                ABBOTTS BOOBY     1.0000    0.8000    0.8889         5
   ABYSSINIAN GROUND HORNBILL     1.0000    1.0000    1.0000         5
        AFRICAN CROWNED CRANE     1.0000    1.0000    1.0000         5
       AFRICAN EMERALD CUCKOO     1.0000    1.0000    1.0000         5
            AFRICAN FIREFINCH     1.0000    1.0000    1.0000         5
       AFRICAN OYSTER CATCHER     1.0000    1.0000    1.0000         5
        AFRICAN PIED HORNBILL     1.0000    1.0000    1.0000         5
          AFRICAN PYGMY GOOSE     1.0000    1.0000    1.0000         5
                    ALBATROSS     1.0000    1.0000    1.0000         5
               ALBERTS TOWHEE     1.0000    1.0000    1.0000         5
         ALEXANDRINE PARAKEET     1.0000    1.0000    1.0000         5
                ALPINE CHOUGH     0.8333    1.0000    0.9091         5
     

### Nhận xét: mô hình performance tốt trên tập Test

Accuracy: Với giá trị accuracy là 0.9832, mô hình của đã phân loại đúng 98.32% các ảnh trong tập dữ liệu test. Đây là một kết quả rất tốt ,mô hình nhận diện và phân loại hình ảnh chim rất tốt.

Macro avg và Weighted avg: Đối với macro avg, precision, recall và F1-score đều cao (lớn hơn 98%). Điều này có nghĩa là mô hình hiệu quả trong việc phân loại các lớp khác nhau của chim một cách đồng đều.

Precision, Recall, và F1-score: Mức độ chính xác, độ phủ và điểm F1 của mô hình đều khá cao, với giá trị lớn hơn 98% => mô hình của không chỉ đạt được độ chính xác cao mà còn giữ được sự cân bằng giữa precision và recall.

f1-score thấp nhất là 0.75 với ALTAMIRA YELLOWTHROAT, AUSTRAL CANASTERO, SAYS PHOEBE, STRIATED CARACARA (trên 525 là chấp nhận được) do có nhiều loại chim có ngoại hình giống nhau chỉ khác biệt chút ít về hình thể hay các chi tiết nhỏ.

=> mô hình thông minh, hoạt động hiệu quả trong việc phân loại hình ảnh các loài chim. 